#**NoteBook Summary**

This notebook processes the UCF-Crime annotated video dataset to create a new dataset of timestamp-aligned image-caption pairs. It begins by loading JSON annotations for training, validation, and test splits, then matches each video key to its corresponding file. For each annotated event in the videos, it samples 1–2 representative frames at fractional intervals between the start and end timestamps and saves them as JPEG images. Captions associated with each event are paired with the extracted frames, and all metadata—such as video name, category, and frame index—is recorded. The final output is a CSV file containing all image paths and their captions (test_image_captions.csv) and a folder of extracted frames (output_frames). The notebook concludes by packaging this data into a structured Kaggle dataset and uploading it using the Kaggle API. This processed dataset is valuable for video-language modeling tasks.

In [3]:
import os
import cv2
import json
import re
import pandas as pd
from collections import defaultdict

# Parameters
ANNOTATION_FILES = [
    '/kaggle/input/ucaucf-crime-annotation-dataset/UCFCrime_Train.json',
    '/kaggle/input/ucaucf-crime-annotation-dataset/UCFCrime_Test.json',
    '/kaggle/input/ucaucf-crime-annotation-dataset/UCFCrime_Val.json'
]
VIDEOS_DIR            = '/kaggle/input/ucaucf-crime-annotation-dataset/UCF_Crimes/UCF_Crimes/Videos'
OUTPUT_DIR            = 'output_frames'
CSV_FILE              = 'test_image_captions.csv'
VIDEO_EXTENSIONS      = ['.mp4', '.avi', '.mov', '.mkv']
DEFAULT_MAX_PER_CAT   = 150
NORMAL_MAX_PER_CAT    = 400
FRAMES_PER_CAPTION    = 2  # Set to 1 or 2

# Helpers
def find_video_file(name, base_dir, exts):
    for root, _, files in os.walk(base_dir):
        for file in files:
            n, ext = os.path.splitext(file)
            if n == name and ext.lower() in exts:
                return os.path.join(root, file)
    return None

def extract_category_from_key(key):
    m = re.match(r'([A-Za-z]+)', key)
    return m.group(1) if m else 'Unknown'

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load and merge annotations
annotations = {}
for fpath in ANNOTATION_FILES:
    with open(fpath, 'r') as f:
        annotations.update(json.load(f))

# Function: sample frames per timestamp interval
def extract_frames_from_timestamps(video_path, video_name, category, timestamps, sentences):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    entries = []

    for idx, ((start_t, end_t), sentence) in enumerate(zip(timestamps, sentences)):
        if end_t <= start_t:
            continue
        # Determine sample fractions for 1 or 2 frames
        if FRAMES_PER_CAPTION == 1:
            fractions = [0.5]
        else:
            fractions = [0.25, 0.75]

        for frac in fractions:
            ts = start_t + frac * (end_t - start_t)
            frame_idx = int(ts * fps)
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
            ret, frame = cap.read()
            if not ret:
                continue

            fname = f"{video_name}_cap{idx}_frame{frame_idx}.jpg"
            path  = os.path.join(OUTPUT_DIR, fname)
            cv2.imwrite(path, frame)
            entries.append({
                'image_path':  path,
                'caption':     sentence,
                'video_key':   video_name,
                'category':    category,
                'frame_index': frame_idx
            })

    cap.release()
    return entries

# Main loop
category_counts = defaultdict(int)
all_entries = []

for vid_key, data in annotations.items():
    cat = extract_category_from_key(vid_key)
    limit = NORMAL_MAX_PER_CAT if cat.lower() == 'normal' else DEFAULT_MAX_PER_CAT
    if category_counts[cat] >= limit:
        continue

    video_file = find_video_file(vid_key, VIDEOS_DIR, VIDEO_EXTENSIONS)
    if not video_file:
        print(f"[WARNING] Missing file for {vid_key}")
        continue

    timestamps = data.get('timestamps', [])
    sentences  = data.get('sentences', [])
    if not timestamps or not sentences:
        continue

    print(f"Processing {vid_key} ({cat})...")
    entries = extract_frames_from_timestamps(
        video_path=video_file,
        video_name=vid_key,
        category=cat,
        timestamps=timestamps,
        sentences=sentences
    )

    all_entries.extend(entries)
    category_counts[cat] += 1
    if category_counts[cat] == limit:
        print(f"Reached {limit} videos for category '{cat}'")

# Save CSV
df = pd.DataFrame(all_entries)
df.to_csv(CSV_FILE, index=False)
print(f"Saved {len(df)} frames to {CSV_FILE}")


Processing Abuse001_x264 (Abuse)...
Processing Abuse002_x264 (Abuse)...
Processing Abuse003_x264 (Abuse)...
Processing Abuse004_x264 (Abuse)...
Processing Abuse005_x264 (Abuse)...
Processing Abuse006_x264 (Abuse)...
Processing Abuse007_x264 (Abuse)...
Processing Abuse008_x264 (Abuse)...
Processing Abuse009_x264 (Abuse)...
Processing Abuse010_x264 (Abuse)...
Processing Abuse011_x264 (Abuse)...
Processing Abuse012_x264 (Abuse)...
Processing Abuse013_x264 (Abuse)...
Processing Abuse014_x264 (Abuse)...
Processing Abuse015_x264 (Abuse)...
Processing Abuse016_x264 (Abuse)...
Processing Abuse017_x264 (Abuse)...
Processing Abuse018_x264 (Abuse)...
Processing Abuse019_x264 (Abuse)...
Processing Abuse020_x264 (Abuse)...
Processing Abuse021_x264 (Abuse)...
Processing Abuse022_x264 (Abuse)...
Processing Abuse023_x264 (Abuse)...
Processing Abuse024_x264 (Abuse)...
Processing Abuse025_x264 (Abuse)...
Processing Abuse026_x264 (Abuse)...
Processing Abuse027_x264 (Abuse)...
Processing Abuse028_x264 (Ab

In [5]:
df

,image_path,caption,video_key,category,frame_index
0,output_frames/Abuse001_x264_cap0_frame39.jpg,"A woman with short hair, slightly fat, wearing...",Abuse001_x264,Abuse,39
1,output_frames/Abuse001_x264_cap0_frame119.jpg,"A woman with short hair, slightly fat, wearing...",Abuse001_x264,Abuse,119
2,output_frames/Abuse001_x264_cap1_frame221.jpg,A man wearing a white shirt and black pants en...,Abuse001_x264,Abuse,221
3,output_frames/Abuse001_x264_cap1_frame243.jpg,A man wearing a white shirt and black pants en...,Abuse001_x264,Abuse,243
4,output_frames/Abuse001_x264_cap2_frame225.jpg,A man wearing a black shirt and black pants en...,Abuse001_x264,Abuse,225
...,...,...,...,...,...
29305,output_frames/Vandalism050_x264_cap0_frame180.jpg,There are three people next to a black car and...,Vandalism050_x264,Vandalism,180
29306,output_frames/Vandalism050_x264_cap1_frame318.jpg,A man in gray clothes with a hat poured someth...,Vandalism050_x264,Vandalism,318
29307,output_frames/Vandalism050_x264_cap1_frame476.jpg,A man in gray clothes with a hat poured someth...,Vandalism050_x264,Vandalism,476
29308,output_frames/Vandalism050_x264_cap2_frame641.jpg,The man in white pants next to the black car p...,Vandalism050_x264,Vandalism,641


In [6]:
!rm -rf test_output_frames


In [7]:
!pip install kaggle --quiet


In [10]:
%%bash
# 1) Create your Kaggle API key file
mkdir -p ~/.kaggle
cat > ~/.kaggle/kaggle.json <<EOF
{
  "username":"nourfakih",
  "key":"0005ac45aa3dc353c01a5d486ed0a5ac"
}
EOF
chmod 600 ~/.kaggle/kaggle.json
echo "✅ ~/.kaggle/kaggle.json created"


✅ ~/.kaggle/kaggle.json created


In [ ]:
# in a notebook cell, once you’ve uploaded kaggle.json via the UI
!mkdir -p /root/.kaggle
!cp kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


In [22]:
# 2) Generate dataset-metadata.json inside your output directory
import os, json

OUTPUT_DIR = '/kaggle/working/'
os.makedirs(OUTPUT_DIR, exist_ok=True)

metadata = {
    "title":       "UCF-Crime Frame+Caption Extractions",
    "id":          "nourfakih/ucf-crime-extracted-frames",
    "subtitle":    "Frames sampled at each annotation timestamp with captions",
    "description": "This dataset contains JPEGs extracted per annotation timestamp from UCF-Crime videos, along with a CSV of matching captions.",
    "licenses": [
        {"name": "CC0-1.0"}
    ]
}

with open(os.path.join(OUTPUT_DIR, 'dataset-metadata.json'), 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✅ {os.path.join(OUTPUT_DIR,'dataset-metadata.json')} written")


✅ /kaggle/working/dataset-metadata.json written


In [17]:
metadata = {
    "title": "UCF-Crime Extracted Frames & Captions",
    "id": "nourfakih/ucf-crime-frames-captions",
    "subtitle": "Frames with timestamp-aligned captions",
    "description": "This dataset contains frames extracted from the UCF-Crime dataset, each matched to timestamped captions. Useful for video-language tasks.",
    "licenses": [{"name": "CC0-1.0"}]
}

with open('/kaggle/working/dataset-metadata.json', 'w') as f:
    json.dump(metadata, f, indent=4)


In [12]:
!kaggle datasets init -p /kaggle/working


Data package template written to: /kaggle/working/dataset-metadata.json


In [14]:
!kaggle datasets create -p  /kaggle/working 


Default slug detected, please change values before uploading


In [19]:
!kaggle datasets create -p /kaggle/working


Skipping folder: .virtual_documents; use '--dir-mode' to upload folders
Starting upload for file test_image_captions.csv
100%|██████████████████████████████████████| 5.00M/5.00M [00:01<00:00, 4.16MB/s]
Upload successful: test_image_captions.csv (5MB)
Skipping folder: output_frames; use '--dir-mode' to upload folders
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/nourfakih/ucf-crime-frames-captions


In [23]:
!kaggle datasets create -p /kaggle/working --dir-mode zip


Starting upload for file .virtual_documents.zip
100%|█████████████████████████████████████████| 22.0/22.0 [00:00<00:00, 63.7B/s]
Upload successful: .virtual_documents.zip (22B)
Starting upload for file test_image_captions.csv
Error while trying to load upload info: ApiStartBlobUploadRequest.__init__() got an unexpected keyword argument 'type'
100%|██████████████████████████████████████| 5.00M/5.00M [00:01<00:00, 4.13MB/s]
Upload successful: test_image_captions.csv (5MB)
Starting upload for file output_frames.zip
100%|████████████████████████████████████████| 845M/845M [00:21<00:00, 41.3MB/s]
Upload successful: output_frames.zip (845MB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/nourfakih/ucf-crime-extracted-frames
